# Appendix

## A.1 Original, Un-optimized Functions

**Toy Example**

In [31]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


**make_word_matrix()**


In [32]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer



#Function to make document, word matricies for LDA#
def make_word_matrix(corpus, needToSplit):
    
    #Define stop words
    stopWords = get_stop_words('english')
    
    
    #Initialize stemmer
    p_stemmer = PorterStemmer()

    
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    
    #Check to make sure NeedToSplit argument is 0 or 1
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    #Check to make sure that dictionary isn't empty#
    if M ==0:
        print("Input dictionary is empty")
        return;
    
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        if isinstance(i, str) != True:
            print("Corpus input is not a string")
            return;
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        # stem tokens
        text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
            #Remove stop words#
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]
            
            #remove stop words
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [33]:
corpusMatrix = make_word_matrix(docs, 1)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference: E-Step** 

In [34]:
import numpy as np
import scipy
from scipy import special

In [35]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]) - 
                                                                                       scipy.special.psi(np.sum(gamma)))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

**Parameter Estimation: M Step**

In [36]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        alphaNew = alphaOld +step
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaOld = alphaNew

    return alphaNew

In [37]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)



**LDA Function:**


In [38]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, corpusMatrix, tol):

    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 1:
        print("Number of topics must be a positive integer greater than 1")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    #alphaOld = 10*np.random.rand(k)
    alphaOld = np.full(shape = k, fill_value = 50/k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [39]:
%%time 
LDA(3, corpusMatrix, 0.1)

CPU times: user 6.96 ms, sys: 81 µs, total: 7.04 ms
Wall time: 7.14 ms


[[[array([[ 0.29731828,  0.35481509,  0.34786662],
          [ 0.42250894,  0.32804881,  0.24944226],
          [ 0.21697984,  0.36641662,  0.41660354]]),
   array([[ 0.40804098,  0.34923284,  0.24272618],
          [ 0.20850419,  0.38813186,  0.40336395],
          [ 0.16578117,  0.64112673,  0.1930921 ],
          [ 0.16578117,  0.64112673,  0.1930921 ],
          [ 0.16578117,  0.64112673,  0.1930921 ],
          [ 0.16578117,  0.64112673,  0.1930921 ]]),
   array([[ 0.45657757,  0.35057942,  0.19284301],
          [ 0.2885281 ,  0.11880568,  0.59266622],
          [ 0.35070078,  0.25410313,  0.39519609]]),
   array([[ 0.29320429,  0.1146013 ,  0.59219441],
          [ 0.45209724,  0.26961047,  0.2782923 ],
          [ 0.46942679,  0.13194139,  0.39863182],
          [ 0.62785292,  0.07622051,  0.29592657],
          [ 0.35768053,  0.24600206,  0.39631742]])],
  [array([ 17.48161878,  17.9483956 ,  17.71115715]),
   array([ 17.82448159,  20.20098668,  18.11570326]),
   array([ 17.64

**Most probable words: mostCommon()**

The following function takes $\beta$, the list of words for a corpus, and the number of words to return for each topic, p, as its inputs. It returns the p words with the most probability for each topic.

In [40]:
#function to return the most probable words
#p is the number of words you want returned for each topic
def mostCommon(beta, wordList, p):
    k = beta.shape[0]
    topicWords = []
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    for i in range(0, k):
        document = betaDF.loc[i,:]
        document.sort(1, ascending = 0)
        mostCommon = pd.DataFrame(document[0:p])
        topicWords.append(mostCommon)
    return(topicWords)

## A.2 Profiling - State of the Union Example

In order to determine the initial performance time of our algorithm, we implemented LDA on a small corpus of eight documents. These documents are each of the State of the Union Adresses (8) delivered by President Clinton during his time in office. These were found in the nltk Python package.

In [41]:
import nltk
from nltk.corpus import state_union

In [42]:
#creating a diciontary of the state of the Union Addresses for Clinton
fileNames = state_union.fileids()
Clinton = fileNames[50:58]

ClintonSOTU = {}
for i in range(0, len(Clinton)):
    ClintonSOTU[Clinton[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(Clinton[i]))

In [43]:
fileNames = state_union.fileids()
SOTU = fileNames

PresSOTU = {}
for i in range(0, len(SOTU)):
    PresSOTU[SOTU[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(SOTU[i]))

In [44]:
%%time
np.random.seed(7)
ClintCorp = make_word_matrix(ClintonSOTU, 0)
Results = LDA(3, ClintCorp, 0.1)

CPU times: user 5min 1s, sys: 1.03 s, total: 5min 2s
Wall time: 5min 3s


In [45]:
! pip install --pre line-profiler
! pip install psutil
! pip install memory_profiler

In [46]:
#creating profile files for the construction of the word matrix and LDA for SOTU example
%prun -q -D ClintonWordMatrix.prof make_word_matrix(ClintonSOTU,0)
%prun -q -D ClintonLDA.prof LDA(3, ClintCorp, 0.1)

 
*** Profile stats marshalled to file 'ClintonWordMatrix.prof'. 
 
*** Profile stats marshalled to file 'ClintonLDA.prof'. 


In [47]:
#printing profile results
import pstats
wordMatrixProfile = pstats.Stats('ClintonWordMatrix.prof')
LDAProfile = pstats.Stats('ClintonLDA.prof')
wordMatrixProfile.print_stats()
LDAProfile.print_stats()
pass

Sat Apr 30 16:44:45 2016    ClintonWordMatrix.prof

         2268672 function calls (2267886 primitive calls) in 2.911 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 {built-in method builtins.sorted}
       16    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
    67104    0.075    0.000    0.916    0.000 /Users/annayanchenko/anaconda3/lib/python3.5/site-packages/nltk/stem/porter.py:631(stem)
    63432    0.075    0.000    0.150    0.000 /Users/annayanchenko/anaconda3/lib/python3.5/site-packages/nltk/stem/porter.py:252(_step1ab)
     5092    0.007    0.000    0.010    0.000 /Users/annayanchenko/anaconda3/lib/python3.5/site-packages/nltk/stem/porter.py:214(_vowelinstem)
        8    0.376    0.047    0.376    0.047 {built-in method numpy.core.multiarray.zeros}
        1    0.000    0.000    0.000    0.000 {built-in method fromkeys}
        1    0.000    

## A.3 Cythonizing

The following code chunks are the Cython versions of the code for updating $\alpha$ and $\beta$. These can be executed in order to examine the areas where the function still heavily relies on Python. We would like to thank Professor Chan for his assistance with the Cython portion of this project.

In [48]:
%reload_ext cython

In [49]:
%%cython -a -lgsl

import cython
import numpy as np
cimport numpy as np
import scipy
from cython_gsl cimport *

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:] cy_alphaUpdate(int k, int M, double[:] alphaOld, double[:, :] gamma, double tol):
    cdef double[:] h = np.zeros(k)
    cdef double[:] g = np.zeros(k)
    cdef double[:] alphaNew = np.zeros(k)

    cdef int converge
    cdef int i, d
    cdef double docSum
    cdef double[:] step = np.zeros(k)
    cdef double c, s1, s2

    cdef double alpha_sum = 0
    for i in range(k):
        alpha_sum += alphaOld[i]

    converge = 0
    while converge == 0:
        z = -gsl_sf_psi_n(1, alpha_sum)

        s1 = 0
        s2 = 1.0/z
        for i in range(0, k):
            docSum = 0
            for d in range(M):
                docSum += gsl_sf_psi(gamma[d,i]) - gsl_sf_psi(sum(gamma[d]))
            g[i] = M*(gsl_sf_psi(alpha_sum) - gsl_sf_psi(alphaOld[i])) + docSum
            h[i] = M*gsl_sf_psi_n(1, alphaOld[i])
            
            s1 += g[i]/h[i]
            s2 += 1.0/h[i]
        c = s1/s2

        for i in range(k):
            step[i] = (g[i] - c)/h[i]
        # step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            for i in range(k):
                alphaNew[i] = alphaOld[i] + step[i]
            alphaOld = alphaNew

    return alphaNew

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:, :] cy_betaUpdate(int k, int M, long[:] phi_1, double[:, :, :] phi_2, double[:, :] gamma, 
                              long[:] c_1, double[:, :, :] c_2, double tol):

    cdef int i, j, d, n
    cdef int Nd
    cdef double wordSum

    #Calculate beta#
    cdef int V = c_2[0].shape[1]
    cdef double[:, :] beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(M):
                Nd = c_1[d] # c[d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi_2[d, n,i]*c_2[d, n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]

    return beta

**Cython Speed-Up**

In [50]:
k = 3
M = 4
phi = [np.array([[ 0.38340797,  0.30327435,  0.31331768],
       [ 0.5703127 ,  0.17742331,  0.25226399],
       [ 0.3176045 ,  0.43558053,  0.24681497]]), np.array([[ 0.55072546,  0.18583053,  0.26344401],
       [ 0.30048528,  0.44698122,  0.2525335 ],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608]]), np.array([[ 0.0623253 ,  0.24856995,  0.68910475],
       [ 0.63063084,  0.08148213,  0.28788703],
       [ 0.12858014,  0.39114008,  0.48027977]]), np.array([[ 0.66366397,  0.08257639,  0.25375964],
       [ 0.77627072,  0.10684157,  0.11688771],
       [ 0.75940435,  0.23665379,  0.00394186],
       [ 0.02553949,  0.62915851,  0.345302  ],
       [ 0.14168349,  0.41504784,  0.44326867]])]
gamma = np.array([np.array([ 9.71859516,  6.63758257,  7.30234283]), np.array([ 10.58244789,   7.50992466,   8.56614802]), np.array([ 9.26880627,  6.44249655,  7.94721775]), np.array([ 10.813832  ,   7.19158249,   7.65310607])])
alphaOld = np.array([ 8.44726999 , 5.72130438,  6.48994619])
c = np.array([np.array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])])
tol = 0.1

r = len(phi)
phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
n = max(phi_1)
p = phi[0].shape[1]
phi_2 = np.zeros((r, n, p))
for i, j in enumerate(phi_1):
    phi_2[i, :j, :] = phi[i]
    
r = len(c)
c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
n = max(c_1)
p = c[0].shape[1]
c_2 = np.zeros((r, n, p))
for i, j in enumerate(c_1):
    c_2[i, :j, :] = c[i]    

In [51]:
%timeit a, b = Mstep(k, 4, phi, gamma, alphaOld, corpusMatrix, tol)

1000 loops, best of 3: 1.35 ms per loop


In [52]:
%%timeit
alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))

The slowest run took 33.53 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 142 µs per loop


In [53]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, corpusMatrix, tol):

    # create rectangular matrices for c
    c = corpusMatrix[0]
    r_c = len(c)
    c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
    n_c = max(c_1)
    p_c = c[0].shape[1]
    c_2 = np.zeros((r_c, n_c, p_c))
    for i, j in enumerate(c_1):
        c_2[i, :j, :] = c[i]
    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        # create rectangular matrices for phi
        r = len(phi)
        phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
        n = max(phi_1)
        p = phi[0].shape[1]
        phi_2 = np.zeros((r, n, p))
        for i, j in enumerate(phi_1):
            phi_2[i, :j, :] = phi[i]

                
        gamma = np.array(gamma)

        alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
        betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [54]:
%%time
LDA(3, corpusMatrix, 0.1)

CPU times: user 3.73 ms, sys: 605 µs, total: 4.33 ms
Wall time: 5.29 ms


[[[array([[ 0.07080204,  0.56166046,  0.36753749],
          [ 0.43285206,  0.505994  ,  0.06115394],
          [ 0.70956438,  0.16928211,  0.1211535 ]]),
   array([[ 0.43215436,  0.49588517,  0.07196048],
          [ 0.69665869,  0.16314571,  0.1401956 ],
          [ 0.23538457,  0.3410982 ,  0.42351722],
          [ 0.23538457,  0.3410982 ,  0.42351722],
          [ 0.23538457,  0.3410982 ,  0.42351722],
          [ 0.23538457,  0.3410982 ,  0.42351722]]),
   array([[  2.88476500e-01,   4.37244826e-01,   2.74278674e-01],
          [  8.25085360e-01,   5.06856795e-04,   1.74407784e-01],
          [  1.17815413e-01,   7.12831334e-01,   1.69353253e-01]]),
   array([[  8.33095579e-01,   5.50292961e-04,   1.66354128e-01],
          [  3.62604533e-01,   5.26772662e-01,   1.10622805e-01],
          [  1.40044410e-01,   7.52843308e-01,   1.07112282e-01],
          [  3.29064686e-01,   6.27744028e-01,   4.31912865e-02],
          [  1.12820515e-01,   7.33982125e-01,   1.53197359e-01]])],
  ar